In [103]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.tsa.api as smt

In [104]:
labels = pickle.load (open('HIST_newdata/output51/label1to21.pkl','rb') )

In [52]:
dataname = 'all_'+'HIST'+''
GRU_1=pickle.load(open('HIST_newdata/output51/'+dataname+'/rolling2009-01-01/pred.pkl.test0','rb'))
GRU_1.columns = ['score1','label1']

In [105]:
GRU_1=pickle.load(open('../TRA/output51/Alpha360_label11/test_pred.pkl','rb'))
# GRU_1.columns = ['score1','label1']
GRU_1

score     label   score_0   score_1   score_2
datetime   instrument                                                  
2019-01-02 SH000300   -0.015012  0.244848 -0.015012  0.196364  0.019862
           SH000905   -0.014663  0.157715 -0.014663  0.224979  0.018708
           SH000906   -0.018329  0.224890 -0.018329  0.205241  0.020559
           SH000985   -0.016305  0.230732 -0.016305  0.212579  0.018887
           SH600000   -0.211410 -0.198604 -0.172697 -0.178636 -0.211410
...                         ...       ...       ...       ...       ...
2022-12-30 SZ301389   -0.334976  0.229846 -0.334976 -0.341819 -0.335525
           SZ301391   -0.352050  1.678695 -0.352050 -0.363011 -0.316488
           SZ301396   -0.185751 -0.656018 -0.185751 -0.184471 -0.146816
           SZ301398   -0.450040  0.297568 -0.450040 -0.464107 -0.452529
           SZ399300    0.201025  1.101686  0.115819  0.201025  0.152544

[4085651 rows x 5 columns]

In [106]:
pred_all = pd.merge(GRU_1.reset_index().drop('label',axis=1), labels.reset_index())

In [107]:
pred_all1 = pred_all.set_index(['datetime','instrument'])
pred_all_norm = pred_all1.groupby(level='datetime', group_keys=False).apply(lambda x:(x-x.mean())/x.std())
# pred_all_norm.corr(method='spearman')*100


In [55]:
with open('acf.txt','a',encoding='utf-8') as f:
    for i in [1,2,6,11,21]:
        rankic = pred_all_norm.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score1'], method='spearman')).mean()
        f.write(str(rankic*100)+'\t')
    f.write('\n')

In [109]:
for i in [6,11,21]:
    if i ==1:
        ic_TRA1 = pred_all_norm.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score'], method='spearman'))
        ic_TRA1 = ic_TRA1.reset_index()
        ic_TRA1.columns = ['date','rankIC'+str(i)]
    elif i ==2:
        ic_TRA2 = pred_all_norm.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score'], method='spearman'))
        ic_TRA2 = ic_TRA2.reset_index()
        ic_TRA2.columns = ['date','rankIC'+str(i)]
        ic_all =   pd.merge(ic_TRA1, ic_TRA2)
    else:
        ic_TRA = pred_all_norm.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score'], method='spearman'))
        ic_TRA = ic_TRA.reset_index()
        ic_TRA.columns = ['date','rankIC'+str(i)]
        ic_all =   pd.merge(ic_all, ic_TRA)

In [113]:
ic_all

,rankIC1,rankIC2,rankIC6,y,rankIC11,date,rankIC21


In [112]:
ic_all.columns = ['date','rankIC6','rankIC11','rankIC21']
ic_all['y'] = ic_all.date.apply(lambda x: str(x)[:4])
ic_all.groupby('y').mean(numeric_only=True)*100

ValueError: Length mismatch: Expected axis has 7 elements, new values have 4 elements

In [114]:
acf1 = []
acf5 = []

stock300 = pred_all_norm.reset_index()['instrument'].drop_duplicates()
lens = len(stock300)
pred_cal = pred_all_norm.reset_index().set_index(['instrument','datetime'])

for a in stock300:
    cal_ser = pred_cal.loc[a]
    if len(cal_ser)>10:
        acf = smt.stattools.acf(cal_ser['score'])
        acf1.append(acf[1])
        acf5.append(acf[5])
    # else:
    #     lens = lens-1
print(np.mean(acf1),np.mean(acf5))
with open('acf.txt','a',encoding='utf-8') as f:
    f.write(str(np.mean(acf1))+'\t')
    f.write(str(np.mean(acf5))+'\t')
    f.write('\n')

0.8023170900265403 0.48751060083242925


In [48]:
datanum = 'csi300_'
labelt = '' #'_label6to1'
with open('HIST_newdata/output6/'+datanum+'GRU'+labelt+'/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
    pred3=pickle.load(f)
pred3.columns = ['score300','label300']

In [49]:
pred_300 = pd.merge(GRU_1.reset_index(), pred3.reset_index())
# pred_300

In [50]:
pred_300_1 = pred_300.set_index(['datetime','instrument'])
pred_300_norm = pred_300_1.groupby(level='datetime', group_keys=False).apply(lambda x:(x-x.mean())/x.std())
# pred_300_norm.corr(method='spearman')*100
pred_300_norm.groupby(level='datetime').apply(lambda x: x['label'+str(300)].corr(x['score1'], method='spearman')).mean()*100

9.124729852633699

In [51]:
ic_TRA = pred_300_norm.groupby(level='datetime').apply(lambda x: x['label'+str(300)].corr(x['score1'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,10.678690
2020,10.326147
2021,9.221801
2022,6.254073


In [95]:
labels6 = pickle.load (open('../KRNN/output/all_label6to1_KRNN/rolling2008-01-01/pred.pkl.test0','rb') )
labels6.columns = ['score6','label6']

In [96]:
labels11 = pickle.load (open('../KRNN/output/all_label11to1_KRNN/rolling2008-01-01/pred.pkl.test0','rb') )
labels11.columns = ['score11','label11']

In [97]:
labels21 = pickle.load (open('../KRNN/output/all_label21to1_KRNN/rolling2008-01-01/pred.pkl.test0','rb') )
labels21.columns = ['score21','label21']

In [98]:
labels_all = pd.merge(labels6.reset_index().drop('score6',axis=1),labels11.reset_index().drop('score11',axis=1))

In [99]:
labels_all = pd.merge(labels_all, labels21.reset_index().drop('score21',axis=1))

In [100]:
all_ind = pd.read_csv('data/cn_data_updated/instruments/all_industry.txt', sep='\t',header=None)
# all_ind

In [101]:
# labels_all = labels_all.reset_index()
all_industry = labels_all[labels_all['instrument'].isin(all_ind[0])] 
all_industry

,datetime,instrument,label6,label11,label21
0,2017-01-03,SH600000,0.338786,1.371176,1.091431
1,2017-01-03,SH600004,0.641504,0.992920,0.850339
2,2017-01-03,SH600006,0.087595,0.547304,0.306904
3,2017-01-03,SH600007,1.094937,1.514964,1.147469
4,2017-01-03,SH600008,0.264073,1.240341,0.763025
...,...,...,...,...,...
6679472,2023-12-29,SZ301518,-0.175927,-0.220445,-0.357976
6679473,2023-12-29,SZ301519,-0.087141,-0.525120,-0.391537
6679474,2023-12-29,SZ301525,-0.630376,0.665941,-0.859407
6679475,2023-12-29,SZ301528,-0.897390,-1.393081,-1.402952


In [102]:
all_industry.set_index(['datetime','instrument']).to_pickle('../KRNN/output/label6to21.pkl')